In [41]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import glob
import os
import rasterio
from rasterio.transform import from_origin
from scipy.interpolate import griddata
import rasterio
from rasterio.transform import from_origin
from pyproj import Transformer
from tqdm import tqdm


In [19]:
#chargement des donnees de temperatures en ile de france

df_temp_77 = pd.read_csv('data/data_temp/Q_75_previous-1950-2023_RR-T-Vent.csv', usecols=["NUM_POSTE", "NOM_USUEL", "AAAAMMJJ", "LAT", "LON", "TNTXM"], sep=";")
df_temp_78 = pd.read_csv('data/data_temp/Q_78_previous-1950-2023_RR-T-Vent.csv', usecols=["NUM_POSTE", "NOM_USUEL", "AAAAMMJJ", "LAT", "LON", "TNTXM"], sep=";")
df_temp_75 = pd.read_csv('data/data_temp/Q_75_previous-1950-2023_RR-T-Vent.csv', usecols=["NUM_POSTE", "NOM_USUEL", "AAAAMMJJ", "LAT", "LON", "TNTXM"], sep=";")
df_temp_91 = pd.read_csv('data/data_temp/Q_91_previous-1950-2023_RR-T-Vent.csv', usecols=["NUM_POSTE", "NOM_USUEL", "AAAAMMJJ", "LAT", "LON", "TNTXM"], sep=";")
df_temp_92 = pd.read_csv('data/data_temp/Q_92_previous-1950-2023_RR-T-Vent.csv', usecols=["NUM_POSTE", "NOM_USUEL", "AAAAMMJJ", "LAT", "LON", "TNTXM"], sep=";")
df_temp_93 = pd.read_csv('data/data_temp/Q_93_previous-1950-2023_RR-T-Vent.csv', usecols=["NUM_POSTE", "NOM_USUEL", "AAAAMMJJ", "LAT", "LON", "TNTXM"], sep=";")
df_temp_94 = pd.read_csv('data/data_temp/Q_94_previous-1950-2023_RR-T-Vent.csv', usecols=["NUM_POSTE", "NOM_USUEL", "AAAAMMJJ", "LAT", "LON", "TNTXM"], sep=";")
df_temp_95 = pd.read_csv('data/data_temp/Q_95_previous-1950-2023_RR-T-Vent.csv', usecols=["NUM_POSTE", "NOM_USUEL", "AAAAMMJJ", "LAT", "LON", "TNTXM"], sep=";")

In [51]:
df_temp = pd.concat([df_temp_77, df_temp_78, df_temp_75, df_temp_91, df_temp_92, df_temp_93, df_temp_94, df_temp_95], ignore_index=True)
df_temp = df_temp.rename(columns={"NUM_POSTE": "id", "NOM_USUEL": "nom", "AAAAMMJJ": "date", "LAT": "latitude", "LON": "longitude", "TNTXM": "temperature"})

In [52]:
df_temp["date"] = pd.to_datetime(df_temp['date'], format='%Y%m%d')
df_temp = df_temp[df_temp["date"].dt.year.isin([2019,2018,2019,2020])]

In [53]:
df_temp

,id,nom,latitude,longitude,date,temperature
55003,75106001,LUXEMBOURG,48.844667,2.333833,2018-01-01,7.4
55004,75106001,LUXEMBOURG,48.844667,2.333833,2018-01-02,9.0
55005,75106001,LUXEMBOURG,48.844667,2.333833,2018-01-03,8.0
55006,75106001,LUXEMBOURG,48.844667,2.333833,2018-01-04,10.5
55007,75106001,LUXEMBOURG,48.844667,2.333833,2018-01-05,8.5
...,...,...,...,...,...,...
3067433,95690001,WY-DIT,49.108167,1.830667,2020-12-27,5.9
3067434,95690001,WY-DIT,49.108167,1.830667,2020-12-28,2.4
3067435,95690001,WY-DIT,49.108167,1.830667,2020-12-29,2.5
3067436,95690001,WY-DIT,49.108167,1.830667,2020-12-30,3.0


In [32]:
# Save the DataFrame to a CSV file
df_temp.to_csv('data/data_temp/temperature_ile_de_france.csv', sep=";", index=False)

In [43]:
df_temp.date.unique()

<DatetimeArray>
['2018-01-01 00:00:00', '2018-01-02 00:00:00', '2018-01-03 00:00:00',
 '2018-01-04 00:00:00', '2018-01-05 00:00:00', '2018-01-06 00:00:00',
 '2018-01-07 00:00:00', '2018-01-08 00:00:00', '2018-01-09 00:00:00',
 '2018-01-10 00:00:00',
 ...
 '2020-12-22 00:00:00', '2020-12-23 00:00:00', '2020-12-24 00:00:00',
 '2020-12-25 00:00:00', '2020-12-26 00:00:00', '2020-12-27 00:00:00',
 '2020-12-28 00:00:00', '2020-12-29 00:00:00', '2020-12-30 00:00:00',
 '2020-12-31 00:00:00']
Length: 1096, dtype: datetime64[ns]

In [49]:
print(df_temp['temperature'].describe())
print(df_temp[df_temp['temperature'] > 60])  # Par exemple, détecter valeurs absurdes


count    49634.000000
mean        13.014750
std          6.629505
min         -6.700000
25%          8.000000
50%         12.500000
75%         18.300000
max         35.000000
Name: temperature, dtype: float64
Empty DataFrame
Columns: [id, nom, latitude, longitude, date, temperature, x, y]
Index: []


In [ ]:
df_temp.to_csv('data/data_temp/temperature_ile_de_france.csv', sep=";", index=False)

In [60]:
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
import rasterio
from rasterio.transform import from_origin
from rasterio.mask import mask
import geopandas as gpd
import os
from tqdm import tqdm
from pyproj import Transformer
from shapely.geometry import mapping

#  Charger les données

df = df_temp[(df_temp['temperature'] > -25) & (df_temp['temperature'] < 60)]

#  Reprojection EPSG:4326 → EPSG:2154
transformer = Transformer.from_crs("EPSG:4326", "EPSG:2154", always_xy=True)
df[['x', 'y']] = df.apply(lambda row: pd.Series(transformer.transform(row['longitude'], row['latitude'])), axis=1)

#  Charger le shapefile de l’Île-de-France en EPSG:2154
gdf = gpd.read_file("data/shapefile/IDF.shp").to_crs("EPSG:2154")
idf_geom = [mapping(geom) for geom in gdf.geometry]

#  Créer le dossier
os.makedirs("rasters_temporels", exist_ok=True)

#  Grille 50 m
res = 50
x_min, y_min, x_max, y_max = gdf.total_bounds

grid_x, grid_y = np.meshgrid(
    np.arange(x_min, x_max, res),
    np.arange(y_min, y_max, res)
)

#  Dates uniques
unique_dates = sorted(df['date'].dt.date.unique())

#  NODATA
nodata_value = -9999

#  Interpolation + masque
with tqdm(total=len(unique_dates), desc="Interpolation avec masque IDF") as pbar:
    for date in unique_dates:
        pbar.set_description_str(f"Traitement : {date}")
        df_day = df[df['date'].dt.date == date]
        if len(df_day) < 5:
            pbar.update(1)
            continue

        points = df_day[['x', 'y']].values
        values = df_day['temperature'].values
        grid_temp = griddata(points, values, (grid_x, grid_y), method='linear')

        if grid_temp is None:
            pbar.update(1)
            continue

        grid_temp = np.where(np.isnan(grid_temp), nodata_value, grid_temp)
        grid_temp = np.clip(grid_temp, -25, 60)

        transform = from_origin(x_min, y_max, res, res)
        tmp_path = f"rasters_temporels/tmp_temp_{date}.tif"
        out_path = f"rasters_temporels/temp_{date}.tif"

        # ➕ Sauvegarde temporaire non-masquée
        with rasterio.open(tmp_path, "w",
                           driver="GTiff",
                           height=grid_temp.shape[0],
                           width=grid_temp.shape[1],
                           count=1,
                           dtype='float32',
                           crs="EPSG:2154",
                           transform=transform,
                           nodata=nodata_value) as dst:
            dst.write(grid_temp.astype('float32'), 1)

        # Appliquer le masque
        with rasterio.open(tmp_path) as src:
            out_image, out_transform = mask(src, idf_geom, crop=True, nodata=nodata_value)
            out_meta = src.meta.copy()
            out_meta.update({
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })

        # 💾 Raster final masqué
        with rasterio.open(out_path, "w", **out_meta) as dest:
            dest.write(out_image)

        os.remove(tmp_path)  # Nettoyage
        pbar.update(1)


C:\Users\lpokambo\AppData\Local\Temp\ipykernel_20196\2786655225.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['x', 'y']] = df.apply(lambda row: pd.Series(transformer.transform(row['longitude'], row['latitude'])), axis=1)
Traitement : 2018-04-10:   9%|▉         | 99/1096 [21:27<3:36:07, 13.01s/it]


RasterioIOError: Attempt to create new tiff file 'rasters_temporels/temp_2018-04-10.tif' failed: No space left on device

In [ ]:
import os
os.environ['REQUESTS_CA_BUNDLE'] = 'H:/PFE Loice/Notebooks/zscaler_bundle.pem'  # adapte le chemin ici

c = cdsapi.Client()

import cdsapi


dataset = "reanalysis-era5-land"  # Corrected dataset name
request = {
    "product_type": "reanalysis",  # Added product type
    "variable": [
        "2m_temperature",
        "soil_temperature_level_1",
        "total_precipitation"
    ],
    "year": "2017",
    "month": "01",
    "day": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12",
        "13", "14", "15",
        "16", "17", "18",
        "19", "20", "21",
        "22", "23", "24",
        "25", "26", "27",
        "28", "29", "30",
        "31"
    ],
    "time": [
        "00:00", "01:00", "02:00",
        "03:00", "04:00", "05:00",
        "06:00", "07:00", "08:00",
        "09:00", "10:00", "11:00",
        "12:00", "13:00", "14:00",
        "15:00", "16:00", "17:00",
        "18:00", "19:00", "20:00",
        "21:00", "22:00", "23:00"
    ],
    "format": "netcdf",  # Corrected key from data_format to format
    "download_format": "unarchived",
    "area": [49.5, 1.5, 48, 3.5]
}

c.retrieve(dataset, request, "output.grib")  # Use the correct client object



2025-05-23 09:06:33,035 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topeaks.
2025-05-23 09:06:33,036 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-23 09:06:33,215 INFO Request ID is 87ef9ce4-8e57-4a64-b058-889e17087461
2025-05-23 09:06:33,298 INFO status has been updated to accepted
2025-05-23 09:08:28,062 INFO status has been updated to successful


74bb5466cea022df975b5da6191e745b.nc:   0%|          | 0.00/1.13M [00:00<?, ?B/s]

'output.grib'

In [ ]:
import os
import cdsapi

os.environ['REQUESTS_CA_BUNDLE'] = 'H:/PFE Loice/Notebooks/zscaler_bundle.pem'

c = cdsapi.Client()

years = ["2017", "2018", "2019"]
months = [f"{m:02d}" for m in range(1, 13)]
days = [f"{d:02d}" for d in range(1, 32)]  # Tous les jours (on filtrera les erreurs)
hours = [f"{h:02d}:00" for h in range(24)]

for year in years:
    for month in months:
        # Créer un nom de fichier unique par mois
        target_file = f"era5_{year}_{month}.nc"

        print(f"Téléchargement de {target_file} en cours...")

        try:
            c.retrieve(
                "reanalysis-era5-land",
                {
                    "product_type": "reanalysis",
                    "variable": [
                        "2m_temperature",
                        "skin_temperature",
                        "surface_net_solar_radiation"
                    ],
                    "year": year,
                    "month": month,
                    "day": days,  # Certains mois n'ont pas 31 jours, ERA5 ignore automatiquement les dates invalides
                    "time": hours,
                    "format": "netcdf",
                    "area": [49.5, 1.5, 48, 3.5]
                },
                target_file
            )
        except Exception as e:
            print(f"Erreur pour {year}-{month} : {e}")


2025-05-23 09:21:32,395 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topeaks.
2025-05-23 09:21:32,395 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


Téléchargement de era5_2017_01.nc en cours...


2025-05-23 09:21:32,656 INFO Request ID is 5201cf18-596a-4bef-a9a4-627c0d5451f2
2025-05-23 09:21:32,742 INFO status has been updated to accepted
2025-05-23 09:21:46,445 INFO status has been updated to running
2025-05-23 09:29:52,372 INFO status has been updated to successful


d168ef53c1410d1fc782dad6ab94de1b.zip:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

Téléchargement de era5_2017_02.nc en cours...


2025-05-23 09:29:54,211 INFO Request ID is d5e4e50f-b141-4f46-95ec-1416b415b21b
2025-05-23 09:29:54,328 INFO status has been updated to accepted
2025-05-23 09:30:27,577 INFO status has been updated to running


In [2]:
import os
import cdsapi
import calendar


os.environ['REQUESTS_CA_BUNDLE'] = 'H:/PFE Loice/Notebooks/zscaler_bundle.pem'

c = cdsapi.Client()

dataset = "reanalysis-era5-land"

years = ["2017", "2018", "2019"]
months = [f"{m:02d}" for m in range(1, 13)]

for year in years:
    for month in months:
        # Calculer le nombre de jours dans le mois (utile pour février et les mois à 30 jours)
        nb_days = calendar.monthrange(int(year), int(month))[1]
        days = [f"{d:02d}" for d in range(1, nb_days + 1)]

        filename = f"era5_land_{year}_{month}.nc"  # extension netcdf

        print(f"Téléchargement : {filename} ...")

        request = {
            "product_type": "reanalysis",
            "variable": [
                "2m_temperature",
                "skin_temperature",
                "surface_net_solar_radiation"
            ],
            "year": year,
            "month": month,
            "day": days,
            "time": [f"{h:02d}:00" for h in range(24)],
            "format": "netcdf",
            "download_format": "unarchived",
            "area": [49.5, 1.5, 48, 3.5]  # ta zone géographique
        }

        c.retrieve(dataset, request, filename)

print("Téléchargements terminés.")


2025-05-23 09:49:26,414 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topeaks.
2025-05-23 09:49:26,414 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


Téléchargement : era5_land_2017_01.nc ...


2025-05-23 09:49:26,812 INFO Request ID is 31b810c4-baf3-4527-94ed-b9bd810eb4a3
2025-05-23 09:49:26,893 INFO status has been updated to accepted
2025-05-23 09:53:45,722 INFO status has been updated to running
2025-05-23 09:59:46,581 INFO status has been updated to successful


58342ac897e3f9cc4be24d251c60b078.nc:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

Téléchargement : era5_land_2017_02.nc ...


...

66f9ad60c8f6c327a91e0f9169872b4.nc:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Téléchargement : era5_land_2019_12.nc ...


2025-05-23 14:10:36,808 INFO Request ID is d45cda20-68d5-4ac3-8eb0-9052a50f98a3
2025-05-23 14:10:36,922 INFO status has been updated to accepted
2025-05-23 14:11:27,594 INFO status has been updated to running
2025-05-23 14:16:58,222 INFO status has been updated to successful


edca26e0addf91d8265aed7b6a56a6fd.nc:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Téléchargements terminés.


In [2]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
import rasterio
from rasterio.transform import from_origin
from rasterio.mask import mask
import geopandas as gpd
import os
from tqdm import tqdm
from pyproj import Transformer
from shapely.geometry import mapping
import warnings
warnings.filterwarnings('ignore')

def idw_interpolation(points, values, grid_points, power=2, smoothing=0, max_distance=None, min_neighbors=3, max_neighbors=12):
    """
    Interpolation par pondération inverse de la distance (IDW) améliorée
    """
    distances = cdist(grid_points, points, metric='euclidean')
    interpolated = np.full(grid_points.shape[0], np.nan)
    
    for i in range(grid_points.shape[0]):
        dist_to_point = distances[i, :]
        
        if max_distance is not None:
            valid_mask = dist_to_point <= max_distance
            if not np.any(valid_mask):
                continue
            dist_to_point = dist_to_point[valid_mask]
            point_values = values[valid_mask]
        else:
            point_values = values
        
        if len(dist_to_point) > max_neighbors:
            nearest_indices = np.argsort(dist_to_point)[:max_neighbors]
            dist_to_point = dist_to_point[nearest_indices]
            point_values = point_values[nearest_indices]
        
        if len(dist_to_point) < min_neighbors:
            continue
        
        dist_to_point = dist_to_point + smoothing
        weights = 1.0 / (dist_to_point ** power)
        weights = weights / np.sum(weights)
        interpolated[i] = np.sum(weights * point_values)
    
    return interpolated

def adaptive_idw_parameters(points, extent, resolution):
    area = (extent[2] - extent[0]) * (extent[3] - extent[1])  # en m²
    density = len(points) / (area / 1e6)  # stations par km²

    if density > 5:
        power = 2.5
        max_distance = 15000
        min_neighbors = 4
        max_neighbors = 8
    elif density > 2:
        power = 2.0
        max_distance = 25000
        min_neighbors = 3
        max_neighbors = 10
    else:
        power = 1.5
        max_distance = 100000
        min_neighbors = 2
        max_neighbors = 20
    
    return power, max_distance, min_neighbors, max_neighbors


# ===== SCRIPT PRINCIPAL =====

df_temp = pd.read_csv("C:/temperature_ile_de_france.csv", sep=";")
df_temp["date"] = pd.to_datetime(df_temp['date'])
df_temp = df_temp[df_temp["date"].dt.year.isin([2017,2019])]
df = df_temp[(df_temp['temperature'] > -25) & (df_temp['temperature'] < 60)].copy()

transformer = Transformer.from_crs("EPSG:4326", "EPSG:2154", always_xy=True)
df[['x', 'y']] = df.apply(lambda row: pd.Series(transformer.transform(row['longitude'], row['latitude'])), axis=1)

gdf = gpd.read_file("data/shapefile/IDF.shp").to_crs("EPSG:2154")
idf_geom = [mapping(geom) for geom in gdf.geometry]

output_dir = "R:/Direction_Data/0_Projets/Projet_CANCAIR/2025_Projet_Loice/temperatures_interpolated_IDW"
os.makedirs(output_dir, exist_ok=True)

res = 50
x_min, y_min, x_max, y_max = gdf.total_bounds
x_coords = np.arange(x_min, x_max, res)
y_coords = np.arange(y_min, y_max, res)
grid_x, grid_y = np.meshgrid(x_coords, y_coords)
grid_points = np.column_stack([grid_x.ravel(), grid_y.ravel()])

unique_dates = sorted(df['date'].dt.date.unique())
nodata_value = -9999

print(f"Traitement de {len(unique_dates)} dates avec interpolation IDW adaptative")

with tqdm(total=len(unique_dates), desc="Interpolation IDW") as pbar:
    for date in unique_dates:
        pbar.set_description_str(f"IDW - {date}")
        out_path = os.path.join(output_dir, f"temp_idw_{date}.tif")
        
        # Vérifier si le fichier existe déjà
        if os.path.exists(out_path):
            pbar.update(1)
            continue
        
        df_day = df[df['date'].dt.date == date]
        if len(df_day) < 2:
            pbar.update(1)
            continue

        points = df_day[['x', 'y']].values
        values = df_day['temperature'].values

        extent = [x_min, y_min, x_max, y_max]
        power, max_distance, min_neighbors, max_neighbors = adaptive_idw_parameters(points, extent, res)
        
        grid_temp_flat = idw_interpolation(
            points=points,
            values=values,
            grid_points=grid_points,
            power=power,
            smoothing=10.0,
            max_distance=max_distance,
            min_neighbors=min_neighbors,
            max_neighbors=max_neighbors
        )

        if np.isnan(grid_temp_flat).sum() > 0:
            print(f"Attention : {np.isnan(grid_temp_flat).sum()} points non interpolés pour la date {date}")
        
        grid_temp = grid_temp_flat.reshape(grid_y.shape)
        grid_temp = np.clip(grid_temp, -25, 60)
        grid_temp = np.where(np.isnan(grid_temp), nodata_value, grid_temp)
        
        transform = from_origin(x_min, y_max, res, res)
        
        tmp_path = os.path.join(output_dir, f"tmp_temp_idw_{date}.tif")
        
        with rasterio.open(tmp_path, "w",
                           driver="GTiff",
                           height=grid_temp.shape[0],
                           width=grid_temp.shape[1],
                           count=1,
                           dtype='float32',
                           crs="EPSG:2154",
                           transform=transform,
                           nodata=nodata_value,
                           compress='lzw') as dst:
            dst.write(grid_temp.astype('float32'), 1)

        with rasterio.open(tmp_path) as src:
            out_image, out_transform = mask(src, idf_geom, crop=True, nodata=nodata_value)
            out_meta = src.meta.copy()
            out_meta.update({
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })

        with rasterio.open(out_path, "w", **out_meta) as dest:
            dest.write(out_image)

        os.remove(tmp_path)
        pbar.update(1)

print(f"Interpolation IDW terminée. Rasters sauvegardés dans : {output_dir}")


Traitement de 730 dates avec interpolation IDW adaptative


IDW - 2019-04-21:  65%|██████▌   | 475/730 [23:02:56<12:22:25, 174.69s/it]    


RasterioIOError: Read failed. See previous exception for details.

In [ ]:
# ===== FONCTION BONUS : VALIDATION CROISÉE =====
def cross_validation_idw(df_day, power=2, max_distance=25000, k_folds=5):
    """
    Validation croisée pour évaluer la qualité de l'interpolation IDW
    """
    from sklearn.model_selection import KFold
    
    points = df_day[['x', 'y']].values
    values = df_day['temperature'].values
    
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    errors = []
    
    for train_idx, test_idx in kf.split(points):
        train_points = points[train_idx]
        train_values = values[train_idx]
        test_points = points[test_idx]
        test_values = values[test_idx]
        
        # Interpoler aux points de test
        predicted = idw_interpolation(
            train_points, train_values, test_points,
            power=power, max_distance=max_distance
        )
        
        # Calculer l'erreur
        valid_mask = ~np.isnan(predicted)
        if np.any(valid_mask):
            mae = np.mean(np.abs(predicted[valid_mask] - test_values[valid_mask]))
            errors.append(mae)
    
    return np.mean(errors) if errors else np.nan

#Exemple d'utilisation de la validation croisée (décommenter pour tester)
test_date = unique_dates[0]
df_test = df[df['date'].dt.date == test_date]
if len(df_test) >= 10:
    mae = cross_validation_idw(df_test)
    print(f"Erreur moyenne absolue (validation croisée) : {mae:.2f}°C")